In [1]:
#!pip3 install torch torchvision torchaudio
%pip install transformers
%pip install PyMuPDF

   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   - -------------------------------------- 0.1/3.2 MB 1.8 MB/s eta 0:00:02
   --- ------------------------------------ 0.3/3.2 MB 2.5 MB/s eta 0:00:02
   ----- ---------------------------------- 0.5/3.2 MB 2.7 MB/s eta 0:00:02
   ------- -------------------------------- 0.6/3.2 MB 2.9 MB/s eta 0:00:01
   --------- ------------------------------ 0.8/3.2 MB 2.7 MB/s eta 0:00:01
   ------------ --------------------------- 1.0/3.2 MB 3.0 MB/s eta 0:00:01
   ------------- -------------------------- 1.0/3.2 MB 3.0 MB/s eta 0:00:01
   --------------- ------------------------ 1.3/3.2 MB 3.0 MB/s eta 0:00:01
   ----------------- ---------------------- 1.4/3.2 MB 3.0 MB/s eta 0:00:01
   ------------------- -------------------- 1.5/3.2 MB 3.0 MB/s eta 0:00:01
   --------------------- ------------------ 1.7/3.2 MB 3.0 MB/s eta 0:00:01
   ---------------------- 

In [2]:
import fitz  # PyMuPDF
from transformers import pipeline, BartForConditionalGeneration, BartTokenizer
import textwrap 


c:\Users\91888\miniconda3\envs\summary\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page_num in range(doc.page_count):
        page = doc[page_num]
        text += page.get_text()
    doc.close()
    return text

def text_summarizer_from_pdf(pdf_path):
    pdf_text = extract_text_from_pdf(pdf_path)

    # Load BART model and tokenizer
    model_name = "facebook/bart-large-cnn"
    model = BartForConditionalGeneration.from_pretrained(model_name)
    tokenizer = BartTokenizer.from_pretrained(model_name)

    inputs = tokenizer.encode("summarize: " + pdf_text, return_tensors="pt", max_length=8048, truncation=True)
    summary_ids = model.generate(inputs, max_length=550, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    formatted_summary = "\n".join(textwrap.wrap(summary, width=80))  # Adjust width as needed
    return formatted_summary

def save_summary_as_pdf(pdf_path, summary):
    # Create a PDF document
    doc = fitz.open()

    page = doc.new_page()
    page.insert_text((20, 50), summary, fontname="helv", fontsize=15)

    output_pdf_path = pdf_path.replace(".pdf", "_summary.pdf")
    doc.save(output_pdf_path)
    doc.close()

    return output_pdf_path

In [6]:
pdf_file_path = "02.intro.news.articles.pdf.pdf"
summary = text_summarizer_from_pdf(pdf_file_path)
output_pdf_path = save_summary_as_pdf(pdf_file_path, summary)
print("Summary saved as PDF:", output_pdf_path)

config.json: 100%|██████████| 1.58k/1.58k [00:00<00:00, 189kB/s]
c:\Users\91888\miniconda3\envs\summary\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\91888\.cache\huggingface\hub\models--facebook--bart-large-cnn. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
model.safetensors: 100%|██████████| 1

Summary saved as PDF: firesafty_summary.pdf
